In [1]:
import socket
import struct
import regex as re
import threading 
import os, sys
import time
import phaseO

MCAST_GRP = '224.0.0.1'
MCAST_PORT = 5050
multiAddrPort = (MCAST_GRP, MCAST_PORT)

IS_MACOS = False
LOCAL_HOST = True

reuse_option = socket.SO_REUSEADDR if not IS_MACOS else socket.SO_REUSEPORT
level_option = socket.IP_ADD_MEMBERSHIP if not LOCAL_HOST else socket.IP_MULTICAST_LOOP

mcast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
mcast_sock.setsockopt(socket.SOL_SOCKET, reuse_option, 1) # for macos, socket.SO_REUSEPORT, linux socket.SO_REUSEADDR

mcast_sock.bind((MCAST_GRP, MCAST_PORT))
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
mcast_sock.setsockopt(socket.IPPROTO_IP, level_option, mreq) # socket.IP_MULTICAST_LOOP for the same machine

uniAddrPort = ("127.0.0.1", 8080)
ucast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ucast_sock.bind(uniAddrPort)

In [2]:
commands = []

multiRunning = []
multiThread = threading.Thread(target=phaseO.mutiListner, name='multicast', 
    args=(multiRunning, commands, mcast_sock, uniAddrPort))
multiThread.daemon = True
multiThread.start()

uniRunning = []
uniThread = threading.Thread(target=phaseO.uniListner, name='unicast', 
    args=(uniRunning, commands, ucast_sock, uniAddrPort))
uniThread.daemon = True
uniThread.start()

globalSeq = [0]
memberNumber = 0
groupMembers = {memberNumber:uniAddrPort}
FTq = phaseO.FTQueue()
messages = {}
seqNumsSent = {}

midRunning = []
midThread = threading.Thread(target=phaseO.middlewareThread, name='middleware', 
    args=(midRunning, globalSeq, memberNumber, groupMembers, 
        FTq, commands, messages, seqNumsSent, 
        ucast_sock, multiAddrPort))
midThread.daemon = True
midThread.start()

Running Multicast Listner
Running Unicast Listner


In [3]:
# joining group
joinM = ('joinRequest', memberNumber)
joinM = ",".join(map(lambda x: str(x),joinM))
ucast_sock.sendto(str.encode(joinM), (MCAST_GRP, MCAST_PORT))

13

In [6]:
print("Group Members:\n{}".format(groupMembers))
print("\nFTq.Q:\n{}".format(FTq.Q))
print("\nClient Messages:\n{}".format(messages))
print("\nSequence Numbers:\n{}".format(seqNumsSent))
print("\nGlobal Sequence Num:\n{}".format(globalSeq))

Group Members:
{0: ('127.0.0.1', 8080), 1: ('127.0.0.1', 8081)}

FTq.Q:
{0: {'que': <queue.Queue object at 0x7fe59d5c1790>, 'label': 12}}

Client Messages:
{}

Sequence Numbers:
{0: '9240915a-35e0-487d-bb50-32fd13f3268f'}

Global Sequence Num:
[1]


In [1]:
FTq.Q[0]['que'].queue

NameError: name 'FTq' is not defined

In [2]:
1

1